In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
import vega_sim.proto.vega as vega_protos

import numpy as np
import time
import vega_sim.proto.data_node.api.v2 as data_node_protos_v2


In [52]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

MM_WALLET = WalletConfig("mm", "mm")

TRADER_1_WALLET = WalletConfig("T1", "pin")

TRADER_2_WALLET = WalletConfig("T2", "pin")

TERMINATE_WALLET = WalletConfig("Settle", "pin")

wallets = [MM_WALLET, TRADER_1_WALLET, TRADER_2_WALLET, TERMINATE_WALLET]

In [137]:
vega = VegaServiceNull(run_with_console=False, use_full_vega_wallet=False, warn_on_raw_data_access=False)
vega.start()

INFO:vega_sim.null_service:Running NullChain from vegahome of /var/folders/yj/cjhtlxn90wldd1hvw5lkxnrc0000gn/T/vega-sim-mg3a57ef
INFO:vega_sim.null_service:Launching GraphQL node at port 52413
INFO:vega_sim.null_service:Launching Console at port 52419


#### Create the wallets

In [138]:
for wallet in wallets:
    vega.create_wallet(wallet.name, passphrase=wallet.passphrase)
vega.forward('10s')

#### Create the settlment asset and mint

In [139]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000,
)

vega.forward("20s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=10e15,  # is this including decimals or not?
)

In [140]:
tdai_id = vega.find_asset_id(symbol="tDAI")

In [141]:
vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=10e7
)
vega.mint(
    TRADER_1_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_2_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.wait_for_total_catchup()

#### Create the market and set up liquidity

In [142]:
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
        position_decimals=2,
        market_decimals=3,
    )

In [143]:
market_id = vega.all_markets()[0].id
print(market_id)

17c33a829b357f5b4df63ef8d46238d97baf1e50e91aeabb91d658a642273c9d


In [144]:
vega.submit_simple_liquidity(
    wallet_name=MM_WALLET.name,
    market_id=market_id,
    commitment_amount=600,
    fee=0.05,
    reference_buy="PEGGED_REFERENCE_MID",
    reference_sell="PEGGED_REFERENCE_MID",
    delta_buy=5,
    delta_sell=5,
    is_amendment=False,
)

vega.forward('1s')
vega.forward('1s')

In [145]:
# Submit orders which will stay on the book and create best bid / ask 
vega.submit_order(
    trading_wallet=MM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=96,
    wait=False,
)

vega.submit_order(
    trading_wallet=MM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=104,
    wait=False,
)



In [146]:
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

mm General = 100000000.0
mm Margin  = 0
mm Bond    = 0
T1 General = 1000000.0
T1 Margin  = 0
T1 Bond    = 0
T2 General = 1000000.0
T2 Margin  = 0
T2 Bond    = 0
Settle General = 0
Settle Margin  = 0
Settle Bond    = 0


#### Do some trading

In [147]:
# Do a trade which will cross
price = 99
vega.submit_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=price,
    wait=False,
)

vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=price,
    wait=False,
)
vega.forward('10s')

# We will now have mark price of 100 


In [148]:
vega.submit_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=10,
    price=price-3,
    wait=False,
)

vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=10,
    price=price+3,
    wait=False,
)

In [149]:
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

mm General = 99999399.98576
mm Margin  = 0.01424
mm Bond    = 600.0
T1 General = 999999.98644
T1 Margin  = 0.01356
T1 Bond    = 0
T2 General = 999999.98644
T2 Margin  = 0.01356
T2 Bond    = 0
Settle General = 0
Settle Margin  = 0
Settle Bond    = 0


In [150]:
vega.submit_liquidity(
        MM_WALLET.name,
        market_id=market_id,
        commitment_amount=2000,
        fee=0.005,
        buy_specs=[
            ("PEGGED_REFERENCE_MID", 1, 1), 
            ("PEGGED_REFERENCE_BEST_BID", 0.4, 1)],
        sell_specs=[
            ("PEGGED_REFERENCE_MID", 1, 1),
            ("PEGGED_REFERENCE_BEST_ASK", 0.4, 1),
        ],
        is_amendment=True,
    )
vega.wait_for_datanode_sync()

In [151]:
num_levels = 11
book_state = vega.market_depth(
            market_id, num_levels=num_levels
        )
bid_prices=[level.price for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_prices=[level.price for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))
bid_volumes=[level.volume for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_volumes=[level.volume for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))

print(ask_prices)
print(ask_volumes)
print(bid_prices)
print(bid_volumes)


[100.0, 102.0, 102.4, 104.0, 0, 0, 0, 0, 0, 0, 0]
[9.48, 10.0, 9.26, 1.0, 0, 0, 0, 0, 0, 0, 0]
[98.0, 96.0, 95.6, 0, 0, 0, 0, 0, 0, 0, 0]
[9.72, 11.0, 9.96, 0, 0, 0, 0, 0, 0, 0, 0]


In [152]:
vega.market_data(market_id)

mark_price: "99000"
best_bid_price: "96000"
best_bid_volume: 1100
best_offer_price: "102000"
best_offer_volume: 1000
best_static_bid_price: "96000"
best_static_bid_volume: 1100
best_static_offer_price: "102000"
best_static_offer_volume: 1000
mid_price: "99000"
static_mid_price: "99000"
market: "17c33a829b357f5b4df63ef8d46238d97baf1e50e91aeabb91d658a642273c9d"
timestamp: 1638185355259066000
open_interest: 100
indicative_price: "0"
market_trading_mode: TRADING_MODE_CONTINUOUS
target_stake: "1089"
supplied_stake: "60000000"
price_monitoring_bounds {
  min_valid_price: "76539"
  max_valid_price: "127702"
  trigger {
    horizon: 86400
    probability: "0.999999"
    auction_extension: 5
  }
  reference_price: "9900000"
}
market_value_proxy: "60000000"
liquidity_provider_fee_share {
  party: "08f96335917e83c4587d2ae08d19332d9e3d00ca71828b77e638fe270ee4ad73"
  equity_like_share: "1"
  average_entry_valuation: "60000000"
  average_score: "1"
}
market_state: STATE_ACTIVE
next_mark_to_market: 1

In [153]:
vega.market_info(market_id).tradable_instrument.log_normal_risk_model

risk_aversion_parameter: 1e-06
tau: 0.00011407711613050422
params {
  sigma: 1.0
}

In [154]:
vega.trading_data_client_v2.GetRiskFactors(data_node_protos_v2.trading_data.GetRiskFactorsRequest(market_id=market_id))

risk_factor {
  market: "17c33a829b357f5b4df63ef8d46238d97baf1e50e91aeabb91d658a642273c9d"
  short: "0.0001140771161305"
  long: "0.0001140771161305"
}

In [155]:
vega.update_market(
    proposal_wallet=MM_WALLET.name,
    market_id=market_id,
    updated_log_normal_risk_model=vega_protos.markets.LogNormalRiskModel(
        risk_aversion_parameter=1e-6,
        tau=0.01,
        params=vega_protos.markets.LogNormalModelParams(sigma=1.0)
    )
)
vega.wait_for_total_catchup()

In [156]:
vega.market_info(market_id).tradable_instrument.log_normal_risk_model

risk_aversion_parameter: 1e-06
tau: 0.01
params {
  sigma: 1.0
}

In [157]:

vega.trading_data_client_v2.GetRiskFactors(data_node_protos_v2.trading_data.GetRiskFactorsRequest(market_id=market_id))

risk_factor {
  market: "17c33a829b357f5b4df63ef8d46238d97baf1e50e91aeabb91d658a642273c9d"
  short: "0.01"
  long: "0.01"
}

In [158]:
vega.update_market(
    proposal_wallet=MM_WALLET.name,
    market_id=market_id,
    updated_log_normal_risk_model=vega_protos.markets.LogNormalRiskModel(
        risk_aversion_parameter=1e-6,
        tau=0.06,
        params=vega_protos.markets.LogNormalModelParams(sigma=1.0)
    )
)
vega.wait_for_total_catchup()

In [159]:
vega.market_info(market_id).tradable_instrument.log_normal_risk_model

risk_aversion_parameter: 1e-06
tau: 0.06
params {
  sigma: 1.0
}

In [160]:
vega.trading_data_client_v2.GetRiskFactors(data_node_protos_v2.trading_data.GetRiskFactorsRequest(market_id=market_id))

risk_factor {
  market: "17c33a829b357f5b4df63ef8d46238d97baf1e50e91aeabb91d658a642273c9d"
  short: "0.06"
  long: "0.06"
}

In [28]:
vega.settle_market(settlement_wallet = TERMINATE_WALLET.name, settlement_price = 95, market_id=market_id)

In [ ]:
vega.forward('1s')

for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

    # positions = vega.positions_by_market(wallet_name=wallet.name, market_id=market_id)
    # print(positions)


In [ ]:
vega.stop()